In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import copy

from imblearn.over_sampling import SMOTE
#시각화
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import *
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss

##알고리즘 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [12]:
# 다양한 오차 측정 지표를 확인하기 위한 함수 정의

from sklearn.metrics import *

def get_clf_eval(y_test, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_score(y_test, pred , average= 'weighted') 
    recall = recall_score(y_test, pred , average= 'weighted') 
    f1 = f1_score(y_test, pred , average= 'weighted') 
    
#     print('정확도(accuracy): {0:.4f}, 정밀도(precision): {1:.4f}, 재현율(recall): {2:.4f}, f1_score: {3:.4f}'.format(accuracy, precision, recall, f1))
    
#     confusion = confusion_matrix(y_test, pred) 
#     accuracy = accuracy_score(y_test, pred) 
#     precision = precision_score(y_test, pred , average= 'macro') 
#     recall = recall_score(y_test, pred , average= 'macro') 
#     f1 = f1_score(y_test, pred , average= 'macro') 
    
#     print('정확도(accuracy): {0:.4f}, 정밀도(precision): {1:.4f}, 재현율(recall): {2:.4f}, f1_score: {3:.4f}'.format(accuracy, precision, recall, f1))
    
    return [accuracy, precision, recall, f1]

In [8]:
def Cut(Series , cuts):
    "구분할 iter가능한 변수와 구분 기준을 입력받으면 구분 기준 앞에서 부터 1로 구분해서 return 함"
    Cuts = copy.deepcopy(cuts)
    Cuts.append(np.inf)
    Cuts.insert(0,-1*np.inf)
    R = len(Cuts)
    label = range(1,R)
    return pd.cut(Series, Cuts, labels = label)

# 데이터 불러오기

In [3]:
df_1_3 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_1.csv")
df_3_6 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_3.csv")
df_0 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_Before_IPO.csv")
df_end = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_EndPrice_IPO.csv")

In [4]:
df_0_1 = df_0.drop(['공모가 대비 3개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_3 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_6 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 3개월 수익률'],axis =1)

In [5]:
df_end_1 = df_end.drop(['종가 대비 3개월 수익률','종가 대비 6개월 수익률','NullCount'],axis =1)
df_end_3 = df_end.drop(['종가 대비 1개월 수익률','종가 대비 6개월 수익률','NullCount'],axis =1)
df_end_6 = df_end.drop(['종가 대비 1개월 수익률','종가 대비 3개월 수익률','NullCount'],axis =1)

# 공통 전처리 및 랜덤포레스트

In [58]:
def process_final(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[-0.4, -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[-0.2, -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1)

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
       
        

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf = RandomForestClassifier(random_state=0 , max_depth = 5 , min_samples_leaf = 3 , min_samples_split =  2 , n_estimators = 400)

        

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train)
        test_pred = clf.predict(X_test)

        # 성과 평가
        train_res = get_clf_eval(y_train, train_pred)
        test_res = get_clf_eval(y_test,test_pred)
        res = train_res + test_res


        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']

        score_list.append(res)
        feature_list.append(feature_importance)
        y_data.append(y_testdata)   
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]*2).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)

    df_feature = pd.DataFrame(feature_list,columns = X_train.columns).T 
    df_feature.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)
        
    return df_score , df_feature , y_data

In [59]:
df_score_0_1,df_feature_0_1,y_data_0_1 = process_final(df_0_1,'공모가 대비 1개월 수익률')

In [60]:
df_score_0_3,df_feature_0_3,y_data_0_3 = process_final(df_0_3,'공모가 대비 3개월 수익률')

In [61]:
df_score_0_6,df_feature_0_6,y_data_0_6 = process_final(df_0_6,'공모가 대비 6개월 수익률')

In [62]:
df_score_1_3,df_feature_1_3,y_data_1_3 = process_final(df_1_3,'1_3_수익률')

In [63]:
df_score_3_6,df_feature_3_6,y_data_3_6 = process_final(df_3_6,'3_6_수익률')

In [64]:
df_score_end_1,df_feature_end_1,y_data_end_1 = process_final(df_end_1,'종가 대비 1개월 수익률')
df_score_end_3,df_feature_end_3,y_data_end_3 = process_final(df_end_3,'종가 대비 3개월 수익률')
df_score_end_6,df_feature_end_6,y_data_end_6 = process_final(df_end_6,'종가 대비 6개월 수익률')

In [80]:
df_score_0_1.to_csv("0_1_모델 평가.csv",encoding = 'euc-kr')
df_score_0_3.to_csv("0_3_모델 평가.csv",encoding = 'euc-kr')
df_score_0_6.to_csv("0_6_모델 평가.csv",encoding = 'euc-kr')
df_score_1_3.to_csv("1_3_모델 평가.csv",encoding = 'euc-kr')
df_score_3_6.to_csv("3_6_모델 평가.csv",encoding = 'euc-kr')
df_score_end_1.to_csv("end_1_모델 평가.csv",encoding = 'euc-kr')
df_score_end_3.to_csv("end_3_모델 평가.csv",encoding = 'euc-kr')
df_score_end_6.to_csv("end_6_모델 평가.csv",encoding = 'euc-kr')

In [81]:
df_feature_0_1.to_csv("0_1_feature별 값.csv",encoding = 'euc-kr')
df_feature_0_3.to_csv("0_3_feature별 값.csv",encoding = 'euc-kr')
df_feature_0_6.to_csv("0_6_feature별 값.csv",encoding = 'euc-kr')
df_feature_1_3.to_csv("1_3_feature별 값.csv",encoding = 'euc-kr')
df_feature_3_6.to_csv("3_6_feature별 값.csv",encoding = 'euc-kr')
df_feature_end_1.to_csv("end_1_feature별 값.csv",encoding = 'euc-kr')
df_feature_end_3.to_csv("end_3_feature별 값.csv",encoding = 'euc-kr')
df_feature_end_6.to_csv("end_6_feature별 값.csv",encoding = 'euc-kr')

In [84]:
frame(y_data_0_1).to_csv("0_1_수익률.csv",encoding = 'euc-kr')
frame(y_data_0_3).to_csv("0_3_수익률.csv",encoding = 'euc-kr')
frame(y_data_0_6).to_csv("0_6_수익률.csv",encoding = 'euc-kr')
frame(y_data_1_3).to_csv("1_3_수익률.csv",encoding = 'euc-kr')
frame(y_data_3_6).to_csv("3_6_수익률.csv",encoding = 'euc-kr')
frame(y_data_end_1).to_csv("end_1_수익률.csv",encoding = 'euc-kr')
frame(y_data_end_3).to_csv("end_3_수익률.csv",encoding = 'euc-kr')
frame(y_data_end_6).to_csv("end_6_수익률.csv",encoding = 'euc-kr')

In [24]:
def buy_sell(y_data):
    portfolio = []
    for i in range(0,32):
        buy = y_data[i][y_data[i]['예측 y 라벨링'] > 3]['실제 y 수익률']
        sell = y_data[i][y_data[i]['예측 y 라벨링'] < 3]['실제 y 수익률']
        sum_1 = buy.sum() - sell.sum()
        sum_2 = len(buy)+len(sell)
        value = sum_1/sum_2
        portfolio.append(value)
    data = pd.DataFrame(portfolio).T
    data.rename(index = {0 : "buy_sell수익률"},columns = lambda x : "model_set_"+ str(x),inplace = True )
    data['mean'] = data.mean(axis=1)[0]
                         
    return data
    

In [25]:
def buy(y_data):
    portfolio = []
    for i in range(0,32):
        buy = y_data[i][y_data[i]['예측 y 라벨링'] > 3]['실제 y 수익률']
        sum_1 = buy.sum()
        sum_2 = len(buy)
        value = sum_1/sum_2
        portfolio.append(value)
    data = pd.DataFrame(portfolio).T
    data.rename(index = {0 : "buy수익률"},columns = lambda x : "model_set_"+ str(x),inplace = True )
    data['mean'] = data.mean(axis=1)[0]
                         
    return data

In [26]:
def mean(y_data):
    portfolio = []
    for i in range(0,32):
        buy = y_data[i]['실제 y 수익률']
        sum_1 = buy.sum()
        sum_2 = len(buy)
        value = sum_1/sum_2
        portfolio.append(value)
    data = pd.DataFrame(portfolio).T
    data.rename(index = {0 : "all수익률"},columns = lambda x : "model_set_"+ str(x),inplace = True )
    data['mean'] = data.mean(axis=1)[0]
    return data

In [27]:
def frame(y_data):
    buy_frame = buy(y_data)
    buy_sell_frame = buy_sell(y_data)
    mean_frame = mean(y_data)
    value = pd.concat([buy_sell_frame,buy_frame,mean_frame])
    return value

In [78]:
def precision(y_data):
    pre = []
    for i in range(0,32):
        new = y_data[i][y_data[i]['예측 y 라벨링'] == 4 | 5]
        real = new[new['실제 y라벨링'] == 4 | 5]
        if len(new) == 0:
            per = 0
        else :
            per = len(real)/len(new)
        pre.append(per)
    data = pd.DataFrame(pre).T
    data.rename(index = {0 : "특이도"},columns = lambda x : "model_set_"+ str(x),inplace = True )
    return data

In [83]:
precision(y_data_0_1).to_csv("0_1_특이도.csv",encoding = 'euc-kr')
precision(y_data_0_3).to_csv("0_3_특이도.csv",encoding = 'euc-kr')
precision(y_data_0_6).to_csv("0_6_특이도.csv",encoding = 'euc-kr')
precision(y_data_1_3).to_csv("1_3_특이도.csv",encoding = 'euc-kr')
precision(y_data_3_6).to_csv("3_6_특이도.csv",encoding = 'euc-kr')
precision(y_data_end_1).to_csv("end_1_특이도.csv",encoding = 'euc-kr')
precision(y_data_end_3).to_csv("end_3_특이도.csv",encoding = 'euc-kr')
precision(y_data_end_6).to_csv("end_6_특이도.csv",encoding = 'euc-kr')